# **Car Price Analysis**

* [Import Packages and Data](#import)
* [Prepare Data](#prepare)

## **Import Packages and Data** <a id='import'></a>

In [2]:
# Import packages
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
# Read in data
cars = pd.read_csv('/Users/kellyshreeve/Desktop/Data-Sets/car_data.csv')

In [4]:
# Print info
print(cars.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [6]:
# Print sample of rows
display(cars.sample(10))

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
244249,14/03/2016 17:44,14300,suv,2014,manual,135,i_reihe,50000,2,petrol,hyundai,NaN,14/03/2016 00:00,0,89335,06/04/2016 00:15
38837,20/03/2016 18:54,1150,sedan,2001,manual,125,vectra,150000,3,petrol,opel,no,20/03/2016 00:00,0,56410,20/03/2016 20:45
69134,22/03/2016 12:54,2299,small,2004,manual,95,forfour,150000,10,petrol,smart,no,22/03/2016 00:00,0,33332,06/04/2016 01:44
280228,26/03/2016 18:48,480,small,1995,manual,70,astra,150000,11,petrol,opel,yes,26/03/2016 00:00,0,56743,06/04/2016 08:17
75331,12/03/2016 16:57,15500,wagon,2014,manual,116,focus,20000,10,gasoline,ford,no,12/03/2016 00:00,0,50354,07/04/2016 07:17
294841,24/03/2016 14:46,6999,sedan,2006,manual,105,a3,150000,11,gasoline,audi,no,24/03/2016 00:00,0,48607,07/04/2016 06:17
66303,12/03/2016 13:38,12600,sedan,2009,manual,143,3er,70000,6,petrol,bmw,no,12/03/2016 00:00,0,57413,04/04/2016 11:46
2213,11/03/2016 19:50,6950,small,2009,auto,90,corsa,50000,4,petrol,opel,no,11/03/2016 00:00,0,86514,23/03/2016 20:17
55783,17/03/2016 10:47,1200,small,1998,NaN,0,fiesta,125000,0,NaN,ford,NaN,17/03/2016 00:00,0,89192,24/03/2016 01:47
47757,04/04/2016 23:50,350,small,1997,manual,45,corsa,150000,2,NaN,opel,no,04/04/2016 00:00,0,55452,05/04/2016 02:41


### Initial data observations

There are 354369 rows and 16 columns in the dataset. Some columns have missing values. All three date columns need to be changed to date time. Column names need to be converted to snake case.

## **Prepare Data** <a id='prepare'></a>

### Fix column names

### Change data types

### Address missing values

### Check for duplicates

## **EDA**

## **Models**